<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/PDF_parsing_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 23 Dec 2023
Aaron, Frank, Sunday, Kennedy and Vishnu.
Contact: vishnu.n@ieee.org

This notebook parses a given PDF,
Creates a parquet file corresponding to it and uploads into HF dataset repo https://huggingface.co/datasets/vishnuramov

Revision history

28 Dec 2023

-

In [ ]:
!pip install pypdf
!pip install langchain
!pip install pandas
!pip install huggingface_hub
!pip install datasets


Here are the Imports

In [7]:
import pandas as pd
from huggingface_hub import login
from google.colab import userdata
import os

Now we read the data from the pdf file.

In [13]:
url = input('Enter the URL: ')
print(f'You entered {url}')

#extract the filename from the URL above.
#attach it to the path as shown below to
#form the colab filename /content/sample_data/<<url filename>>
#then go get it.
input_path, input_name = os.path.split(url)
filename = '/content/sample_data/'+input_name
print(filename)

Enter the URL: https://www.itu.int/en/ITU-T/focusgroups/an/Documents/Use-case-AN.pdf
You entered https://www.itu.int/en/ITU-T/focusgroups/an/Documents/Use-case-AN.pdf
https://www.itu.int/en/ITU-T/focusgroups/an/Documents
/content/sample_data/Use-case-AN.pdf


In [15]:
os.system('wget {} -O {}'.format(url, filename))


0

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(filename)
pages = loader.load_and_split()
simple_data_container = []

for i in range(len(pages)):
  print("Physical Page =",
      i,
      "logical page given by the loader = ",
      pages[i].metadata['page'],
      "of ",
      pages[i].metadata['source'],
      "content = ",
      pages[i].page_content)
  simple_data_container.append(
      [i,
       pages[i].metadata['source'],
       pages[i].page_content])
  print("*****************************")

df = pd.DataFrame(simple_data_container,
                  columns=['page_number', 'document_source', 'page_content'])
df.head(10)

#cut the extension from the filename.
#attach .parquet to it
#then write the parquet file.
parquet_filename = os.path.splitext(filename)[0]+'.parquet'
print(parquet_filename)


In [18]:
df.to_parquet(parquet_filename)

*access* the HF hub,
upload the dataset into it


In [19]:
hf_token=userdata.get('my_hf_write_token')
login(hf_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


CAUTION: Make sure the dataset exists before proceeding.

Go to https://huggingface.co/vishnuramov

and check if the dataset

vishnuramov/ITU-AN-usecases-1 exists.

If not, one should create it.

In [21]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj=parquet_filename,
    path_in_repo=parquet_filename,
    repo_id="vishnuramov/ITU-AN-usecases-1",
    repo_type="dataset",
)
print("Check https://huggingface.co/vishnuramov")

Use-case-AN.parquet:   0%|          | 0.00/122k [00:00<?, ?B/s]

Check https://huggingface.co/vishnuramov


TBD: Create a notebook which reads

dataset(s) from the HF dataset repo https://huggingface.co/datasets/vishnuramov:
and analyses it .